In [1]:
import requests
import json
from random import randint
from tqdm import trange
import os
import pandas as pd
import re
from groq import Groq
import csv
import random

In [3]:
def get_output(message):
    linshi=random.randint(0,4)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": message,
            }
        ],
        model="llama3-70b-8192",
        temperature=0
    )
    return chat_completion.choices[0].message.content

In [4]:
def get_output1(message):
    linshi=random.randint(0,4)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Output format: Ans: A or B",
            },
            {
                "role":"user",
                "content":message,
            }
        ],
        model="llama3-70b-8192",
        temperature=0
    )
    return chat_completion.choices[0].message.content

In [5]:
def get_bias_output(ans,fact):
    bias1=''
    bias2=''
    if ans=='0':
        bias1='If you choice "Ans: A" and it is wrong, there will be severe punishment.'
        bias2='If you choice "Ans: B" and it is wrong, there will be no punishment.'
    if ans=='1':
        bias1='If you choice "Ans: A" and it is wrong, there will be no punishment.'
        bias2='If you choice "Ans: B" and it is wrong, there will be severe punishment.'
    template="""
Here is a content that may be biased by cognitive bias. Rewrite it such that a reviewer is not biased.

Prompt:Please help me classify the following economic phenomenon.     
%s    
%s

Economic Phenomenon:%s

There is no need to output which sentences that have deleted or changed.
The options will be given later and are not the missing part."""%(bias1,bias2,fact)
    return template

In [6]:
jishu=0
ff= open('../data/test.jsonl','r')
for i in trange(500):
    data=json.loads(ff.readline())
    temp1=get_bias_output(str(data['ans']),data['question'])
    #print(temp1)
    ans1=get_output(temp1)
    while 'This rewritten' in ans1:
        ans1=re.findall(r"([\S|\s]*)This rewritten([\S|\s]*)",ans1, re.S|re.I)[0][0]
    #print(ans1)
    while "Note" in ans1:
        ans1=re.findall(r"([\S|\s]*)Note([\S|\s]*)",ans1, re.S|re.I)[0][0]
    while "(" in ans1:
        ans1=re.findall(r"([\S|\s]*)\(([\S|\s]*)",ans1, re.S|re.I)[0][0]
    while "By" in ans1:
        ans1=re.findall(r"([\S|\s]*)By([\S|\s]*)",ans1, re.S|re.I)[0][0]
    while "I made" in ans1:
        ans1=re.findall(r"([\S|\s]*)I made([\S|\s]*)",ans1, re.S|re.I)[0][0]
    try:
        ans1=re.findall(r"Prompt:([\S|\s]*)",ans1, re.S|re.I)[0]
    except:
        ans1=re.findall(r"bias:([\S|\s]*)",ans1, re.S|re.I)[0]
    temp2=ans1
    ans2=get_output1(temp2+'\nA.Expansionary economic phenomenon   B.Contractionary economic phenomenon')
    with open("self_help.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'methods':ans2,'ans':str(data['ans']),'question':temp2}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')

100%|██████████| 500/500 [17:37<00:00,  2.11s/it]
